In [1]:
import sys
import tensorflow as tf
sys.path.append("/visuworks/Blindless_AIFFELTON/Ablation Study/")

# import customized modules
import preprocess, data_generator, check_result, metrics, loss, train

sys.path.append("/visuworks/Blindless_AIFFELTON/Ablation Study/Pyramid")
import pyramid


# Insert Path
# {model_name}_{epoch}ep_{model_inpusize}_{Generator | aG/fG/sG}
FILE_NAME = 'pyramid_30ep_512_aG_wo_decay'
INPUT_SHAPE = (512, 512, 1)
MODEL_PATH = "/visuworks/Blindless_AIFFELTON/Ablation Study/Pyramid/model_parameters/" + FILE_NAME + ".tf"
HISTORY_PATH = "/visuworks/Blindless_AIFFELTON/Ablation Study/Pyramid/history/" + FILE_NAME + '_history' ".json"

TRAIN_SOURCE = '/visuworks/Dataset/Selected Dataset 3/Train'
TEST_SOURCE = '/visuworks/Dataset/Selected Dataset 3/Test'

2024-01-31 13:34:08.781678: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-31 13:34:08.781703: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-31 13:34:08.782739: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-31 13:34:08.788021: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-31 13:34:09.362517: W tensorflow/compiler/tf2

In [2]:
# Define custom objects for loading the model
custom_objects = {'DiceLoss': loss.DiceLoss(), 
                  'sensitivity': metrics.sensitivity,
                  'specificity': metrics.specificity,
                  'accuracy' : metrics.accuracy}


# Create augmentation
train_preproc = preprocess.build_augmentation_for_adaptive()
test_preproc = preprocess.build_augmentation_for_adaptive(is_train=False)

# Create train, test generator
train_generator = data_generator.AdaptiveDataGenerator(
    TRAIN_SOURCE, 
    number_of_images = 2,
    img_size=INPUT_SHAPE,
    output_size=INPUT_SHAPE,
    is_train=True,
    augmentation= train_preproc
)

test_generator = data_generator.AdaptiveDataGenerator(
    TRAIN_SOURCE, 
    number_of_images = 2,
    img_size=INPUT_SHAPE,
    output_size=INPUT_SHAPE,
    is_train=True,
    augmentation= train_preproc
)

# Create model
model = pyramid.build_model(INPUT_SHAPE)
model.summary()

2024-01-31 13:34:10.533333: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-31 13:34:10.566265: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-31 13:34:10.566456: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 512, 512, 1)]        0         []                            
                                                                                                  
 conv2d_2 (Conv2D)           (None, 512, 512, 16)         32        ['input_1[0][0]']             
                                                                                                  
 conv2d (Conv2D)             (None, 512, 512, 8)          16        ['input_1[0][0]']             
                                                                                                  
 conv2d_1 (Conv2D)           (None, 512, 512, 16)         160       ['input_1[0][0]']             
                                                                                              

In [3]:
train.model_train(model,
                  epoch = 30,
                  train_generator = train_generator,
                  test_generator = test_generator,
                  model_path = MODEL_PATH,
                  history_path = HISTORY_PATH,
                  save_every_epoch=True,
                  apply_weight_decay=False)

Epoch 1/30


2024-01-31 13:34:16.084769: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2024-01-31 13:34:18.755902: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f301f370f50 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-01-31 13:34:18.755923: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3080 Ti, Compute Capability 8.6
2024-01-31 13:34:18.759247: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1706708058.831360   16065 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


440/440 [==============================] - ETA: 0s - loss: 0.7025 - sensitivity: 0.0778 - specificity: 0.9782 - accuracy: 0.9165INFO:tensorflow:Assets written to: /visuworks/Blindless_AIFFELTON/Ablation Study/Pyramid/model_parameters/pyramid_30ep_512_aG_wo_decay.tf/1--0.7025-0.0778-0.9782-0.3116-0.9834.pb/assets


INFO:tensorflow:Assets written to: /visuworks/Blindless_AIFFELTON/Ablation Study/Pyramid/model_parameters/pyramid_30ep_512_aG_wo_decay.tf/1--0.7025-0.0778-0.9782-0.3116-0.9834.pb/assets


Model saved to /visuworks/Blindless_AIFFELTON/Ablation Study/Pyramid/model_parameters/pyramid_30ep_512_aG_wo_decay.tf/1--0.7025-0.0778-0.9782-0.3116-0.9834.pb
440/440 [==============================] - 192s 398ms/step - loss: 0.7025 - sensitivity: 0.0778 - specificity: 0.9782 - accuracy: 0.9165 - val_loss: 0.5209 - val_sensitivity: 0.3116 - val_specificity: 0.9834 - val_accuracy: 0.9413
Epoch 2/30
440/440 [==============================] - ETA: 0s - loss: 0.4492 - sensitivity: 0.5433 - specificity: 0.9518 - accuracy: 0.9261INFO:tensorflow:Assets written to: /visuworks/Blindless_AIFFELTON/Ablation Study/Pyramid/model_parameters/pyramid_30ep_512_aG_wo_decay.tf/2--0.4492-0.5433-0.9518-0.4863-0.9732.pb/assets


...

Model saved to /visuworks/Blindless_AIFFELTON/Ablation Study/Pyramid/model_parameters/pyramid_30ep_512_aG_wo_decay.tf/28--0.2877-0.6768-0.9766-0.7507-0.9633.pb
440/440 [==============================] - 185s 421ms/step - loss: 0.2877 - sensitivity: 0.6768 - specificity: 0.9766 - accuracy: 0.9579 - val_loss: 0.3043 - val_sensitivity: 0.7507 - val_specificity: 0.9633 - val_accuracy: 0.9519
Epoch 29/30
439/440 [============================>.] - ETA: 0s - loss: 0.2827 - sensitivity: 0.6770 - specificity: 0.9780 - accuracy: 0.9599INFO:tensorflow:Assets written to: /visuworks/Blindless_AIFFELTON/Ablation Study/Pyramid/model_parameters/pyramid_30ep_512_aG_wo_decay.tf/29--0.2826-0.6772-0.9780-0.7079-0.9779.pb/assets


INFO:tensorflow:Assets written to: /visuworks/Blindless_AIFFELTON/Ablation Study/Pyramid/model_parameters/pyramid_30ep_512_aG_wo_decay.tf/29--0.2826-0.6772-0.9780-0.7079-0.9779.pb/assets


Model saved to /visuworks/Blindless_AIFFELTON/Ablation Study/Pyramid/model_parameters/pyramid_30ep_512_aG_wo_decay.tf/29--0.2826-0.6772-0.9780-0.7079-0.9779.pb
440/440 [==============================] - 182s 414ms/step - loss: 0.2826 - sensitivity: 0.6772 - specificity: 0.9780 - accuracy: 0.9599 - val_loss: 0.2687 - val_sensitivity: 0.7079 - val_specificity: 0.9779 - val_accuracy: 0.9621
Epoch 30/30
440/440 [==============================] - ETA: 0s - loss: 0.2754 - sensitivity: 0.6863 - specificity: 0.9770 - accuracy: 0.9588INFO:tensorflow:Assets written to: /visuworks/Blindless_AIFFELTON/Ablation Study/Pyramid/model_parameters/pyramid_30ep_512_aG_wo_decay.tf/30--0.2754-0.6863-0.9770-0.6899-0.9789.pb/assets


INFO:tensorflow:Assets written to: /visuworks/Blindless_AIFFELTON/Ablation Study/Pyramid/model_parameters/pyramid_30ep_512_aG_wo_decay.tf/30--0.2754-0.6863-0.9770-0.6899-0.9789.pb/assets


Model saved to /visuworks/Blindless_AIFFELTON/Ablation Study/Pyramid/model_parameters/pyramid_30ep_512_aG_wo_decay.tf/30--0.2754-0.6863-0.9770-0.6899-0.9789.pb
440/440 [==============================] - 187s 426ms/step - loss: 0.2754 - sensitivity: 0.6863 - specificity: 0.9770 - accuracy: 0.9588 - val_loss: 0.2704 - val_sensitivity: 0.6899 - val_specificity: 0.9789 - val_accuracy: 0.9614
